In [ ]:
import os
import copy
from ego4d.internal.colmap.preprocess import (
    ColmapConfig,
    produce_colmap_script,
    get_colmap_data_dir,
)

%matplotlib inline
import matplotlib.pyplot as plt

from matplotlib import rcParams

import cv2

# Generate COLMAP inputs + Validate

Pleae generate the configuration as appropriate, and check the visualizations below under "Validate Configuration".

Once you have generated a validation configuration, you may upload this to S3. Please refer to the "Upload Configuration" section.

## Configure + Generate

Please refer to the inline documentation below.

In [ ]:
COLMAP_BIN = None  # supply your COLMAP_BIN path here
VRS_BIN = None  # supply your path to VRS here

# be sure to use an absolute path
DATA_DIR = "/private/home/miguelmartin/ego4d/ego4d_public/data_experiments_test/"

In [ ]:
config = ColmapConfig(
    # what take to use
    uni_name="unc",
    take_id="T1",
    
    # where to save data
    data_dir=DATA_DIR,
    # whether we want COLMAP to use the GPU whenever possible
    use_gpu=True,
    
    # whether we are taking synchronized exo-centric frames
    sync_exo_views=True,
    
    # there are three rot_mode's:
    # - 0 => perform no rotation
    # - 1 => rotate aria to exo
    # - 2 => rotate exo to aria
    rot_mode=1,
    
    # refer to https://colmap.github.io/cameras.html
    # OPENCV_FISHEYE is for GoPro
    # THIN_PRISM_FISHEYE is for Aria
    camera_model="OPENCV_FISHEYE",
#     camera_model="THIN_PRISM_FISHEYE",
    
    # the inverse of the number of frames per second to sample
    # the mobile walkaround and aria walkaround
    frame_rate=0.25,
    
    # specific mobile frames to use
    # if None then all frames are considered using `frame_rate`
    mobile_frames=None,
    
    # whether to include the aria walkaround
    include_aria=True,
    # assuming the walkaround is in the VRS file, when is the last timestamp that the walkaround occurs
    # if not provided will use `aria_frames` if aria_use_sync_info is False
    aria_last_walkthrough_sec=None,
    # if aria_use_sync_info is True then aria_last_walkthrough_sec will be assigned 
    # based off timesync data
    aria_use_sync_info=True,
    # specific frames for the walkaround 
    # if not provided will use all frames in aria will be used (subsampled using `frame_rate`)
    aria_frames=None,
    
    # where to sample the exo videos from
    exo_from_frame=700,
    exo_to_frame=720,
    # specific frames for each exo video
    # if one is provided below, all must be provided and exo_from_frame/exo_to_frame must be None
    exo1_frames=None,
    exo2_frames=None,
    exo3_frames=None,
    exo4_frames=None,
    
    # the name of the configuration (a relative path/directory where all frames and config is saved to)
    # if none a name will be automatically constructed from the above configuration
    name=None,
    
    # misc
    aria_fps=30,      # aria is assumed to be (approx) 30fps by default
    exo_fps=None,     # if none, this will be determined from the video file
    mobile_fps=None,  # if none, this will be determined from the video file
    run_colmap=False, # whether we want to run colmap after
    colmap_bin=COLMAP_BIN, # where colmap is located, if None a default will be provided
    vrs_bin=VRS_BIN,       # where vrs is located, if None a default will be provided
    download_video_files=True, # whether we stream from S3 or download videos (changeme if timeout occurs)
    force_download=False, # if download_video_files is True, force_download will force a re-download of video files
)

In [ ]:
produce_colmap_script(copy.deepcopy(config))

# Validate Configuration

The below section visualizes the exo-centric frames. Please verify that there is no occulusions in each frame. If there is an occulusion, such as a QR code, this may cause issues with SfM (COLMAP).

In [ ]:
root_dir = get_colmap_data_dir(config)
frames_dir = os.path.join(root_dir, "frames")
exo_img_paths = {
    x: [
        os.path.join(frames_dir, x, p)
        for p in os.listdir(os.path.join(frames_dir, x))
    ]
    for x in os.listdir(frames_dir)
    if x.startswith("cam")
}

N_frames = max(len(v) for _, v in exo_img_paths.items())
N_cams = len(exo_img_paths)
sorted_is = sorted(exo_img_paths.items(), key=lambda x: x[0])
viz = [
    (cam_i, frame_i, name + f"_frame_{frame_i}", path)
    for cam_i, (name, paths) in enumerate(sorted_is)
    for frame_i, path in enumerate(paths)
]

fig = plt.figure(figsize=(N_cams * 4, N_frames * 4))
ax = fig.subplots(N_cams, N_frames)
plt.subplots_adjust(left=0, bottom=0, right=1, top=1, wspace=0, hspace=0.1)

for i, j, name, path in viz:
    x = cv2.imread(path)
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    h, w = x.shape[0:2]
    ax[i][j].set_xticks([], [])
    ax[i][j].set_yticks([], [])
    ax[i][j].imshow(x)
    ax[i][j].set_title(name)

# Upload Configuration

Please upload your configuration to S3

In [ ]:
# The location is to be determined, please hold off on uploading for now.